In [2]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [3]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


68.0 x 143 x 19.0


In [4]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [5]:
if "Alarm" in cnc.status:
    cnc.home()

In [6]:
cnc.init()

True

# Generate Program


In [7]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = GCode.GCode(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

def dot_grid(XX=(0, 10), YY=(0, 10), Z=0, power=1, dwell=1, **kwargs):
    XXX, YYY = np.meshgrid(XX, YY)
    program = GCode.GCode(**kwargs)
    
    for XXX_, YYY_ in zip(XXX, YYY):
        for X, Y in zip(XXX_, YYY_):
            program+=dot(X=X, Y=Y, Z=Z, power=power, dwell=dwell)
    return program

In [19]:
spacings=[0.05, 0.1, 0.2, 0.5, 1, 2]
dwell=0.5
Z=-14

cnc.init()
for idx2, power in enumerate([100, 125, 175, 200, 225, 255]):
    Y=idx2*5+15
    for idx, spacing in enumerate(spacings):
        XX=np.arange(idx*5, (idx+1)*10, spacing)
        for X in XX:
            program=dot(X=X, Y=Y, Z=Z, power=power, dwell=dwell)
            cnc.run(program)

In [ ]:
X0=10
XX=np.arange(X0, X0+5, spacing)
XX

In [ ]:
Y0=10
YY=np.arange(Y0, Y0+5, spacing)
YY

In [ ]:
len(range(10,BlockLength-10, 5))

In [ ]:
cnc.init()
for X0 in range(10,BlockLength-10, 5):
    XX=np.arange(X0, X0+5, spacing)
    program = dot_grid(XX=XX, YY=YY, Z=-14, power=100, dwell=0.5)
    cnc.run(program)

In [ ]:
program = dot_grid(XX=XX, YY=YY, Z=-14, power=255, dwell=0.5)

In [ ]:
program.machine=cnc

In [ ]:
program.run()